# Import

In [1]:
# pip install plotly
# pip install cufflinks
# pip install chart_studio

In [57]:
import plotly.offline
import plotly.express as px # express 통해서 그리기
import chart_studio.plotly as py
import plotly.graph_objects as go
import datetime

# - Standard plotly imports
import cufflinks as cf
cf.go_offline(connected=True)
p

In [4]:
import pandas as pd
kobis_tot = pd.read_csv("https://raw.githubusercontent.com/yujin-jo/mini-project-team1/main/1.%20Crawling/data/KOBIS_TOT.csv")


## 2011년~2020년까지 독립영화 수의 추세가 늘고 있을 것이다

### 1) 전체 영화에서 독립/예술영화 비중 구하기

In [5]:
# 독립/예술영화 데이터만 수집
# 독립영화 : 제작사나 투자자들의 자본과 지원을 받지 않고 만드는 영화
art = kobis_tot[(kobis_tot.영화구분 == '독립/예술영화')]
no_art = kobis_tot[(kobis_tot.영화구분 != '독립/예술영화')]
art
no_art

# 독립/예술영화 개수
art_cnt = art.shape[0] # 1515개
no_art_cnt = no_art.shape[0] # 1860개
# tmp 만들기
tmp = pd.DataFrame({'분야':['일반영화', '독립/예술영화'], '영화수':[no_art_cnt, art_cnt]})
# 그래프 구하기
fig = px.pie(tmp, values='영화수', names='분야', title='전체 영화에서 독립/예술영화의 비중'
            ,color_discrete_sequence=px.colors.sequential.Plasma
            ,width=600, height=600)
fig.update_traces(textposition="inside", textinfo="percent+label")

### 2) 독립/예술영화 분야의 장르 구하기

In [6]:
# 독립/예술영화 장르
art_genre = art.groupby('장르')['영화명'].count().reset_index()
art_genre.columns = ['장르', '영화수']
art_genre

fig = px.pie(art_genre, values='영화수', names='장르', title='독립/예술영화 장르에 따른 비중'
            ,color_discrete_sequence=px.colors.sequential.Plasma
            ,width=600, height=600)
fig.update_traces(textposition="inside", textinfo="percent+label")
# plotly.offline.plot(fig) # 새로운 창에서 열 수 있음
fig.show()

### 3) 개봉연도별 독립/예술영화 추세 구하기

In [45]:
# 년도별 독립/예술영화 개수
movie_cnt = kobis_tot.groupby(['개봉연도', '영화구분'])['영화명'].count().reset_index()
movie_cnt.columns = ['개봉연도', '영화구분', '영화수']
movie_cnt
# 영화 구분에 따른 산점도
fig = px.scatter(movie_cnt, x='개봉연도', y='영화수'
          , color= '영화구분' # 영화구분에 따라서 색깔표현
          , size = '영화수' # 영화수에 따라서 크기 변화
          , title = '개봉연도별 영화 구분에 따른 산점도'
          , log_x=True) # 하위 항목 다 보이게 설정
fig.show()

In [78]:
# 개봉연도별 독립영화 관객수나 매출액으로 추세 살펴보기
# 본 사람이 없는데 영화수만 늘어난 건 아닐까?
m_customer = kobis_tot[(kobis_tot.영화구분 == '독립/예술영화')].groupby(['개봉연도'])[['전국관객수']].sum().reset_index()
m_money = kobis_tot[(kobis_tot.영화구분 == '독립/예술영화')].groupby(['개봉연도'])[['전국매출액']].sum().reset_index()

m_tot = kobis_tot[(kobis_tot.영화구분 == '독립/예술영화')].groupby(['개봉연도'])[['전국매출액', '전국관객수']].sum().reset_index()
fig = px.bar(m_tot, x='개봉연도', y='전국매출액', title='개봉연도별 독립영화 매출액 추세', log_x=True)
fig.show()

In [56]:
fig = px.bar(m_customer, x='개봉연도', y='전국관객수', title='개봉연도별 독립영화 관객수 추세', log_x=True)
fig.show()

In [83]:
m_screen = kobis_tot.groupby(['개봉연도', '영화구분'])[['전국스크린수']].sum().reset_index()
fig = px.bar(m_screen, x='개봉연도', y='전국스크린수', color='영화구분', title='개봉연도별 독립영화 전국스크린 수 추세', log_x=True)
fig.show()

In [54]:
### 2020년은 왜 갑자기 훅 떨어졌나?
# 그리고 독립영화는 어떤 형태로 제작이 되어가는지
# 일반 상업영화는 어떤 형태로 제작이 되어가길래

### 4) 개봉 월별 독립/예술영화 추세 구하기

In [46]:
# 개봉 월별 독립/예술영화 개수
movie_month_cnt = kobis_tot.copy()
movie_month_cnt['개봉일'] = pd.to_datetime(movie_month_cnt['개봉일'])
# 개봉월 컬럼 추가
movie_month_cnt['개봉월'] = movie_month_cnt['개봉일'].dt.month 
movie_month_cnt = movie_month_cnt.groupby(['개봉연도', '개봉월', '영화구분'])['영화명'].count().reset_index()
# 개봉연월 추가
movie_month_cnt['개봉연월'] = movie_month_cnt["개봉연도"].map(str) + " " + movie_month_cnt["개봉월"].map(str)
# 독립/예술영화 부족한 데이터 추가
add_data1 = {'개봉연도': 2012, '개봉월': 1, '영화구분':'독립/예술영화', '영화명':0, '개봉연월':'2012 1'}
add_data2 = {'개봉연도': 2011, '개봉월': 12, '영화구분':'독립/예술영화', '영화명':0, '개봉연월':'2011 12'}
movie_month_cnt = movie_month_cnt.append(add_data1, ignore_index=True)
movie_month_cnt = movie_month_cnt.append(add_data2, ignore_index=True)
# 정렬 수정
movie_month_cnt = movie_month_cnt.sort_values(by=["개봉연도",'개봉월'])
movie_month_cnt
# 필요없는 컬럼 삭제
movie_month_cnt = movie_month_cnt.drop(['개봉연도', '개봉월'], axis=1)
movie_month_cnt.columns = ['영화구분', '영화수', '개봉연월']
# 개봉월별에 따른 산점도
# fig = px.scatter(movie_month_cnt, x='개봉연월', y='영화수'
#           , color= '영화구분' # 영화구분에 따라서 색깔표현
#           , title = '개봉 월별 영화 구분에 따른 산점도')

# fig.show()

fig = px.bar(movie_month_cnt, x='개봉연월', y='영화수', color='영화구분', title='개봉 월별 영화 구분에 따른 그래프')
fig.show()


## 개봉연도별 독립/예술영화의 스크린수 비중과 매출액 순위 / 관객 수 확인

### 1) 독립/예술영화와 스크린 수의 비중 구하기

In [10]:
# 개봉연도별 독립/예술영화 스크린 수
movie_screen_cnt = kobis_tot.groupby(['영화구분'])['전국스크린수'].sum().reset_index()
movie_screen_cnt

fig = px.pie(movie_screen_cnt, values='전국스크린수', names='영화구분', title='독립/예술영화 스크린수 비중'
            ,color_discrete_sequence=px.colors.sequential.Plasma
            ,width=600, height=600)
fig.update_traces(textposition="inside", textinfo="percent+label")

### 2) 독립/예술영화의 스크린 수에 따른 매출액 확인

In [ ]:
kobis_tot.groupby(['영화구분'])['전국스크린수'].sum().reset_index()

# 독립영화수가 